## Julia on Colab Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools https://github.com/Dale-Black/DistanceTransforms.jl.git"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.1 on the current Colab Runtime...
2021-06-22 23:59:14 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.1-linux-x86_64.tar.gz [112784227/112784227] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package https://github.com/Dale-Black/DistanceTransforms.jl.git...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.6.1
Commit 6aaedecc44 (2021-04-23 05:59 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 2


## Import Packages

In [2]:
using BenchmarkTools
using DistanceTransforms

## Load test data

In [3]:
x = rand([0,1], (112, 112, 96, 2, 1))
y = rand([0,1], (112, 112, 96, 2, 1))

x_dtm = DistanceTransforms.compute_dtm(x)
y_dtm = DistanceTransforms.compute_dtm(y);

## Vanilla dice loss

In [4]:
@benchmark DistanceTransforms.dice_loss($x, $y)

BenchmarkTools.Trial: 
  memory estimate:  55.13 MiB
  allocs estimate:  9
  --------------
  minimum time:     21.255 ms (0.00% GC)
  median time:      23.975 ms (9.45% GC)
  mean time:        23.417 ms (6.59% GC)
  maximum time:     26.540 ms (11.59% GC)
  --------------
  samples:          214
  evals/sample:     1

## Vanilla Hausdorff loss

In [5]:
@benchmark DistanceTransforms.hd_loss($x, $y, $x_dtm, $y_dtm)

BenchmarkTools.Trial: 
  memory estimate:  18.38 MiB
  allocs estimate:  7
  --------------
  minimum time:     12.174 ms (0.00% GC)
  median time:      12.824 ms (0.00% GC)
  mean time:        13.402 ms (4.30% GC)
  maximum time:     17.231 ms (17.67% GC)
  --------------
  samples:          373
  evals/sample:     1

In [6]:
@benchmark DistanceTransforms.hd_loss($x, $y, $DistanceTransforms.compute_dtm(x), $DistanceTransforms.compute_dtm(y))

BenchmarkTools.Trial: 
  memory estimate:  1.36 GiB
  allocs estimate:  24328272
  --------------
  minimum time:     2.586 s (12.90% GC)
  median time:      2.633 s (13.15% GC)
  mean time:        2.633 s (13.15% GC)
  maximum time:     2.681 s (13.39% GC)
  --------------
  samples:          2
  evals/sample:     1

## Parallelized dice loss

In [7]:
@benchmark DistanceTransforms.dice_lossP($x, $y)

BenchmarkTools.Trial: 
  memory estimate:  2.12 KiB
  allocs estimate:  42
  --------------
  minimum time:     5.198 ms (0.00% GC)
  median time:      5.521 ms (0.00% GC)
  mean time:        5.548 ms (0.00% GC)
  maximum time:     9.679 ms (0.00% GC)
  --------------
  samples:          898
  evals/sample:     1

## Parallelized Hausdorff loss

In [8]:
@benchmark DistanceTransforms.hd_lossP($x, $y, $x_dtm, $y_dtm)

BenchmarkTools.Trial: 
  memory estimate:  2.09 KiB
  allocs estimate:  41
  --------------
  minimum time:     6.076 ms (0.00% GC)
  median time:      6.365 ms (0.00% GC)
  mean time:        6.377 ms (0.00% GC)
  maximum time:     7.766 ms (0.00% GC)
  --------------
  samples:          782
  evals/sample:     1

In [9]:
@benchmark DistanceTransforms.hd_lossP($x, $y, $DistanceTransforms.compute_dtm(x), $DistanceTransforms.compute_dtm(y))

BenchmarkTools.Trial: 
  memory estimate:  1.34 GiB
  allocs estimate:  24328308
  --------------
  minimum time:     2.676 s (13.55% GC)
  median time:      2.682 s (13.79% GC)
  mean time:        2.682 s (13.79% GC)
  maximum time:     2.688 s (14.02% GC)
  --------------
  samples:          2
  evals/sample:     1